In [ ]:
%%bash 
pip3 install dask[complete] toolz cloudpickle

In [ ]:
from __future__ import division
import scipy.stats as stats
import pandas as pd
import dask.dataframe as dd
import numpy as np
import math
import time
import sys
import csv
import os

In [310]:
directory = "CDR_Data_IC/SET1TSV/"
ant_pos_loc = "CDR_Data_IC/ANT_POS.TSV"
all_CDR_files = "CDR_Data_IC/SET1TSV/*.TSV"
vor_path = "CDR_Data_IC/Voronoi_Polygons/Proportions/"
points_in_polygon = "CDR_Data_IC/Points_In_Polygon/"

# Retrieve required files for Cote D'Ivoire.
complete_cdr_data = dd.read_csv(all_CDR_files, delim_whitespace=True, header=None, 
                       names=['Date', 'Time', 'Outgoing', 'Terminating', 'Num_Calls', 'Total_Duration'],
                       usecols=['Outgoing', 'Terminating', 'Num_Calls'])

antenna_positions = pd.read_csv(ant_pos_loc, delim_whitespace=True, header=None, 
                                names = ["CT", "Latitude", "Longitude"], index_col="CT")

voronoi_admin_1 = pd.read_csv(vor_path + "voronoi_admin_1.csv", 
                              usecols=["CT", "O_Area_km2", "ID_1", "P_Area_km2"], index_col="CT")

voronoi_admin_2 = pd.read_csv(vor_path + "voronoi_admin_2.csv", 
                              usecols=["CT", "O_Area_km2", "ID_2", "P_Area_km2"], index_col="CT")

voronoi_admin_3 = pd.read_csv(vor_path + "voronoi_admin_3.csv", 
                              usecols=["CT", "O_Area_km2", "ID_3", "P_Area_km2"], index_col="CT")

pip_admin_1 = pd.read_csv(points_in_polygon + "PIP_Admin_1.csv", usecols=["ID_1", "NUMPOINTS"], index_col="ID_1")
pip_admin_1.index.names = ['Region']
pip_admin_1.sort_index(ascending=True, inplace=True)

pip_admin_2 = pd.read_csv(points_in_polygon + "PIP_Admin_2.csv", usecols=["ID_2", "NUMPOINTS"], index_col="ID_2")
pip_admin_2.index.names = ['Region']
pip_admin_2.sort_index(ascending=True, inplace=True)

pip_admin_3 = pd.read_csv(points_in_polygon + "PIP_Admin_3.csv", usecols=["ID_3", "NUMPOINTS"], index_col="ID_3")
pip_admin_3.index.names = ['Region']
pip_admin_3.sort_index(ascending=True, inplace=True)

In [ ]:
def findSameGeoLocatedCTs(CT_pos):
    ids = list(CT_pos.index.unique())
    id_series = CT_pos.index.unique()

    # The CTs which have the same geo-location.
    result = CT_pos[CT_pos.duplicated(["Latitude","Longitude"], keep=False)].sort_values("Latitude")
    return result, ids

same_geo, ids = findSameGeoLocatedCTs(antenna_positions)
print(same_geo)

In [ ]:
def findMissingCTs(CDR_Data):
    present_ids = np.unique(CDR_Data[["Outgoing", "Terminating"]].values)
    result = set(ids) - set(present_ids)
    return result

start = time.time()
missing_data = findMissingCTs(complete_cdr_data)
elapsed = time.time() - start
print(missing_data)
print(elapsed / 60)


In [ ]:
def computeUndirectedWeights(dataframe):
    a = dataframe.copy()
    b = dataframe.copy()

    c = pd.merge(a, b, left_on=['Terminating','Outgoing'], right_on=['Outgoing', 'Terminating'], how='left')
    c = c.fillna(0)
    c['Weight'] = 0

    # If outgoing != incoming then weight = call_a + call_b, else use call_a only.
    c['Weight'] = (c['Num_Calls_x'] + c['Num_Calls_y']).where(c['Outgoing_x'] != c['Terminating_x'], c['Num_Calls_x'])
        
    c.drop(['Outgoing_y', 'Terminating_y'], axis=1, inplace=True)
    c = c.rename(columns={'Outgoing_x': 'Outgoing', 'Terminating_x': 'Terminating', 'Num_Calls_x': 'Num_Calls'})
    c.index.name = 'Index'
    
    return c
    
def preprocessCDRData(CDR_Data):

    keep_columns = ['Outgoing', 'Terminating', 'Num_Calls']
    CDR_Data = CDR_Data[keep_columns]
    
    # Remove any row that has an unknown CT.
    known_outgoing = CDR_Data['Outgoing'] >= 0
    known_terminating = CDR_Data['Terminating'] >= 0
    CDR_Data = CDR_Data[known_outgoing & known_terminating]

    # Initial Grouping to get all existing CT pairs.
    grouped_1 = CDR_Data.groupby(by=['Outgoing', 'Terminating'])
    
    # Sum up Total Number of Calls per (Outgoing, Terminating) pair. 
    all_pairs = grouped_1.sum().compute()
    all_pairs.reset_index(inplace=True)
    all_pairs.index.names = ['Index']
    print("Total number of calls between CT pairs\n")
    print(all_pairs.head())
    print()
    
    # Compute undirected weight of (Outgoing, Terminating) pair.
    bidirectional = computeUndirectedWeights(all_pairs)
    bidirectional.reset_index(inplace=True)
    print("Bidirectional CT Pairs")
    print(bidirectional.head())
    print()
    
    # Group by Outgoing CT.
    CT_grouping = bidirectional.groupby('Outgoing')
    
    # Total Activity for each CT.     
    total_activity = CT_grouping.sum()
    total_activity.reset_index(inplace=True)
    total_activity.set_index('Outgoing', inplace=True)
    total_activity= total_activity.rename(columns={'Weight': 'Total_Weight'}) 
    keep_columns=['Total_Weight']
    total_activity =  total_activity[keep_columns]
    
    print("CTs with total activity")
    print(total_activity.head())
    print()
    
    return bidirectional, CT_grouping, total_activity
    

start = time.time()
bidirectional_CT, CT_groups, total_CT_activity = preprocessCDRData(complete_cdr_data)
elapsed = time.time() - start
print(elapsed / 60)

In [ ]:
# METRIC 1: Activity.

def computeActivity(voronoi_data, CT_activity, region_name, scale):
    regional_activity = pd.DataFrame(columns=['Region', 'Activity'])
    regional_activity['Region'] = list(voronoi_data[region_name].unique())
    regional_activity.set_index('Region', inplace=True)
    regional_activity['Activity'] = 0
    regional_activity.sort_index(ascending=True, inplace=True)
    
    indices = CT_activity.index
    
    for index, row in voronoi_data.iterrows():
        ct = index
        region = row[region_name]
        
        if(row['O_Area_km2'] == 0):
            continue
        proportion = row['P_Area_km2'] / row['O_Area_km2']
        
        # Proportion = nan sometimes because original area = 0.
        if(math.isnan(proportion)):
            continue

        if(ct in indices):
            prev_value = regional_activity.loc[region, 'Activity']
            ct_weight = CT_activity.loc[ct, 'Total_Weight'] * scale
            regional_activity.loc[region, 'Activity'] = (proportion * ct_weight) + prev_value
    
    return regional_activity

regional_activity_1 = computeActivity(voronoi_admin_1, total_CT_activity, "ID_1", 1)
regional_activity_2 = computeActivity(voronoi_admin_2, total_CT_activity, "ID_2", 1)
regional_activity_3 = computeActivity(voronoi_admin_3, total_CT_activity, "ID_3", 1)

print(regional_activity_1)

In [ ]:
# METRIC 3: Network Advantage.

def extractKMedian(ct_activity, grouping):
    ct_activity['kmedian'] = 0
    ct_activity['k_degree'] = 0
    ct_activity['k_sum'] = 0
    
    for name, group in grouping:
        weights = sorted(list(group['Weight']))
        
        mid = int(len(weights) / 2)
        k_mid = weights[mid]
        ct_activity.loc[name, 'kmedian'] = k_mid
        
        k_deg = sum(1 for _ in weights if _ > k_mid)
        ct_activity.loc[name, 'k_degree'] = k_deg
        
        k_sum = sum(x for x in weights if x > k_mid)
        ct_activity.loc[name, 'k_sum'] = k_sum
        
def computeNormalisedEntropy(ct_activity, grouping, ct_pairs, voronoi_data, region_data, region_name):
    
    # Merge CT total activity with pairs to give CT totals & K Medians for purpose of calculating the fractions.    
    merged = pd.merge(ct_pairs, ct_activity, left_on = 'Outgoing', right_index=True)
    keep_columns = ['Index', 'Outgoing', 'Terminating', 'Weight', 'Total_Weight']
    merged =  merged[keep_columns]
    merged.set_index('Index', inplace=True)
    
    # Calculate fraction * log(fraction) per CT pair.     
    fraction = merged['Weight'] / merged['Total_Weight']
    natural_log_fraction = np.log(fraction)
    merged['Fraction_Product'] = fraction * natural_log_fraction

    print("Merged CT_pairs & Total_Activity")
    print(merged.head())
    print()
    
    # Get total of fraction * log(fraction) for each CT. [CT, Sum_Fraction_Product].   
    sum_fraction_product = merged.groupby('Outgoing').sum()
    keep_columns = ['Fraction_Product']
    sum_fraction_product = sum_fraction_product[keep_columns]
    print(sum_fraction_product.head())
    print()
    
    # Merge ct_activity with fraction_product. Add entropy column = sum_fraction_product / log(k_degree).  
    ct_activity = pd.merge(ct_activity, sum_fraction_product, how='left', left_index=True, right_index=True)
    ct_activity['ct_entropy'] = (-1 * (ct_activity['Fraction_Product']) / np.log(ct_activity['k_degree']))
    print(ct_activity.head())
    print()
    
    indices = ct_activity.index
    
    region_entropy = region_data.copy()
    region_entropy['entropy'] = 0
    
    for index, row in voronoi_data.iterrows():
        ct = index
        region = row[region_name]
        
        if(row['O_Area_km2'] == 0):
            continue
            
        proportion = row['P_Area_km2'] / row['O_Area_km2']
        
        # Proportion = nan sometimes because original area = 0. An approximation as actual value is very small.
        if(math.isnan(proportion)):
            continue

        if(ct in indices):
            prev_value = region_entropy.loc[region, 'entropy']
            new_value = ct_activity.loc[ct, 'ct_entropy']
            region_entropy.loc[region, 'entropy'] = (proportion * new_value) + prev_value
    
    # Normalise entropy by the number of CTs in the region.
    region_entropy['normalised_entropy'] = region_entropy['entropy'] / region_entropy['NUMPOINTS']
    return region_entropy

def computeMedianDegree(ct_activity, ct_pairs, voronoi_data, region_data, region_name, scale):
    total_activity = ct_activity.copy()
    
    keep_columns = ['k_sum']
    total_activity = total_activity[keep_columns]
    total_activity.rename(columns={'k_sum' : 'Total_Weight'}, inplace=True)
    
    print(total_activity.head())
    regional_activity = computeActivity(voronoi_data, total_activity, region_name, scale)
    print(regional_activity.head())
    
    regional_activity = pd.merge(regional_activity, region_data, left_index=True, right_index=True)
    regional_activity['medDegree'] = regional_activity['Activity'] / regional_activity['NUMPOINTS']
    print(regional_activity)
    keep_columns = ['medDegree']
    regional_activity = regional_activity[keep_columns]
    
    return regional_activity

extractKMedian(total_CT_activity, CT_groups)

# Normalised Entropy
norm_entropy_1 = computeNormalisedEntropy(total_CT_activity, CT_groups, bidirectional_CT, 
                                          voronoi_admin_1, pip_admin_1, "ID_1")

norm_entropy_2 = computeNormalisedEntropy(total_CT_activity, CT_groups, bidirectional_CT, 
                                          voronoi_admin_2, pip_admin_2, "ID_2")

norm_entropy_3 = computeNormalisedEntropy(total_CT_activity, CT_groups, bidirectional_CT, 
                                          voronoi_admin_3, pip_admin_3, "ID_3")

# Median Degree.
medDegree_1 = computeMedianDegree(total_CT_activity, bidirectional_CT, voronoi_admin_1, pip_admin_1, "ID_1", 1)
medDegree_2 = computeMedianDegree(total_CT_activity, bidirectional_CT, voronoi_admin_2, pip_admin_2, "ID_2", 1)
medDegree_3 = computeMedianDegree(total_CT_activity, bidirectional_CT, voronoi_admin_3, pip_admin_3, "ID_3", 1)

In [340]:
# METRIC 4: Introversion.

def computeIntroversion(ct_activity, ct_pairs, voronoi_data, region_data, region_name):
    ct_data = ct_pairs.copy()
    
    keep_columns = ['Outgoing', 'Terminating', 'Weight']
    ct_data = ct_data[keep_columns]
    ct_data['Inner_Traffic'] = (ct_data['Weight']).where(ct_data['Outgoing'] == ct_data['Terminating'], 0)
    
    ct_data = ct_data.groupby('Outgoing').sum()
    keep_columns = ['Weight', 'Inner_Traffic']
    ct_data = ct_data[keep_columns]
    
    ct_data['Outer_Traffic'] = ct_data['Weight'] - ct_data['Inner_Traffic']
    ct_data['Introversion'] = ct_data['Inner_Traffic'] / ct_data['Outer_Traffic']
    
    print(ct_data.head())
    
    indices = ct_activity.index
    
    region_introversion = region_data.copy()
    region_introversion['Introversion'] = 0
    
    for index, row in voronoi_data.iterrows():
        ct = index
        region = row[region_name]
        
        if(row['O_Area_km2'] == 0):
            continue
            
        proportion = row['P_Area_km2'] / row['O_Area_km2']
        
        # Proportion = nan sometimes because original area = 0. An approximation as actual value is very small.
        if(math.isnan(proportion)):
            continue

        if(ct in indices):
            prev_value = region_introversion.loc[region, 'Introversion']
            new_value = ct_data.loc[ct, 'Introversion']
            region_introversion.loc[region, 'Introversion'] = (proportion * new_value) + prev_value
    
    region_introversion['avg_introversion'] = region_introversion['Introversion'] / region_introversion['NUMPOINTS']
    return region_introversion
    

start = time.time()

introversion_1 = computeIntroversion(total_CT_activity, bidirectional_CT, voronoi_admin_1, pip_admin_1, "ID_1")
introversion_2 = computeIntroversion(total_CT_activity, bidirectional_CT, voronoi_admin_2, pip_admin_2, "ID_2")
introversion_3 = computeIntroversion(total_CT_activity, bidirectional_CT, voronoi_admin_3, pip_admin_3, "ID_3")

print(introversion_1)

elapsed = time.time() - start
print(elapsed / 60)

             Weight  Inner_Traffic  Outer_Traffic  Introversion
Outgoing                                                       
1          707770.0        90981.0       616789.0      0.147507
2          159785.0        11706.0       148079.0      0.079052
3          174330.0        18274.0       156056.0      0.117099
4          238144.0        16061.0       222083.0      0.072320
5         3084195.0       368007.0      2716188.0      0.135487
        NUMPOINTS  Introversion  avg_introversion
Region                                           
1              42      9.321934          0.221951
2               9      3.097162          0.344129
3             106     28.715430          0.270900
4              12      3.490770          0.290897
5              34      7.840913          0.230615
6              44      9.121727          0.207312
7              68     14.807226          0.217753
8              56      7.953796          0.142032
9             463     33.413921          0.072168
10

In [ ]:
# METRIC 2.
def calculate_distance(lat_a, long_a, lat_b, long_b):
    earth_radius_km = 6371.0
    
    lat1 = radians(lat_a)
    lon1 = radians(long_a)
    lat2 = radians(lat_b)
    lon2 = radians(long_b)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(sqrt(a), math.sqrt(1 - a))
    distance = R * c
    print("Result:", distance)
    return distance

# function to calculate the population of each CT voronoi.

# gravity residual: population in each CT voronoi, calculate distances between each pair
def computeGravityResidual(CT_pairs, outgoing_CTs, terminating_CTs, voronoi_data):
    #     create a data frame containing each pair and the distance and pop_a * pop_b and flow estimate, actual & diff
    #     I need the total activity of each pair - get it from complete_outgoing & terminating
    #     Remember I need to plot, so will need another function.
    print()

